<a href="https://colab.research.google.com/github/juanpajedrez/pytorch_learning/blob/main/08_pytorch_paper_replicating_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08. Milestone Paper Replication: PyTorch Replicating Paper

The goal of machine learning research paper replicating is: turn an ML research paper into usable code.

In this notebook, we're going to be replicating the Vision Transformer (ViT) architecture with PyTorch:

Paper link: https://arxiv.org/pdf/2010.11929
Paper Abs: https://arxiv.org/abs/2010.11929


## 0. Get Setup
Let's import code we've previously written + required libraries.

In [ ]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
  import torch
  import torchvision
  assert int(torch.__version__.split(".")[1]) >= 12 or int(torch.__version__.split(".")[0]) == 2, "torch version should be 1.12+"
  assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")
except:
  print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
  !pip3 install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  import torch
  import torchvision
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")

In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
!nvidia-smi

## 1. Get data

The whole goal of what we're trying to do is to replicate the ViT transformer architecture for our FoodVision Mini problem.

To do that, we need some data.

Namely, the pizza, steak and sushi images we've been using so far.

In [ ]:
# Download pizza, steak, sushi images from GitHub
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

In [ ]:
# Setup directory paths to train and test images
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
train_dir, test_dir

## 2. Create Datasets and DataLoaders

In [ ]:
from torchvision import transforms
from going_modular.going_modular import data_setup

# Create image size
IMG_SIZE = 224 # come from Table 3 of the ViT paper.

# Create transforms pipeline
manual_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

print(f"Manually created transforms: {manual_transform}")

In [ ]:
# Create a batch size of 32 (the paper uses 4096 but this may be too big for our smaller hardware... we scale regarding our hardware).
BATCH_SIZE = 32

# Create DataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
  train_dir = train_dir,
  test_dir = test_dir,
  transform = manual_transform,
  batch_size = BATCH_SIZE
)

len(train_dataloader), len(test_dataloader), class_names

### 2.3 Visualize a single image

As always, let's adhere to the motto *visualize, visualize, visualize!*

In [ ]:
# Get a batch of images
image_batch, label_batch = next(iter(train_dataloader))
image, label = image_batch[0], label_batch[0]

# View the single image and label shapes
image.shape, label

In [ ]:
# Plot the image with matplotlib
import matplotlib.pyplot as plt

plt.imshow(image.permute(1, 2, 0)) # (color_channels, height, width) -> (height, width, color_channels)
plt.title(class_names[label])
plt.axis(False)

## 3. Replicating ViT: Overview

Looking at a whole machine learning research paper can be intimidating.

So in order to make it more undeerstandable, we can break it down into smaller pieces.

* **Inputs:** - What goes into the model? (in our case, image tensors)
* **Outputs:** - What comes out of the model/layer/block> (in our case, we want the model to output image classification labels).
* **Layers** - Takes an input, manipulates it with a function (for example could be self-attention).
* **Blocks** - A collection of layers:.
* **Model** -  A collection of blocks.

### 3.1 Overview: Pieces of the puzzle.

* Figure 1: Visual Overview of the architecture
* Four Equations: Math equations which define the function of layer/block.
* Table 1/3: Different hyperparameters for the architecture/training.
* Text.


### Figure 1

![](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/images/08-vit-paper-figure-1-architecture-overview.png?raw=true)

* Embedding = learnable representation (start with random numbers and improve them over time)

### Four equations

![](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/images/08-vit-paper-four-equations.png?raw=true)

**Equation 1:**
An overview of the model is depicted in Figure 1. The standard Transformer receives as input a 1D sequence of token embeddings. To handle 2D images, we reshape the image $\mathbf{x} \in \mathbb{R}^{H \times W \times C}$ into a sequence of flattened 2D patches $\mathbf{x}_p \in \mathbb{R}^{N \times\left(P^2 \cdot C\right)}$, where ( $H, W$ ) is the resolution of the original image, $C$ is the number of channels, $(P, P)$ is the resolution of each image patch, and $N=H W / P^2$ is the resulting number of patches, which also serves as the effective input sequence length for the Transformer. The Transformer uses constant latent vector size $D$ through all of its layers, so we flatten the patches and map to $D$ dimensions with a trainable linear projection (Eq. 1). We refer to the output of this projection as the patch embeddings.

**Equation 1:**
Position embeddings are added to the patch embeddings to retain positional information. We use standard learnable 1D position embeddings, since we have not observed significant performance gains from using more advanced 2D-aware position embeddings (Appendix D.4). The resulting sequence of embedding vectors serves as input to the encoder.

```python
# Equation 1
x_input = [class_token, image_patch_1, image_patch_2, ..., image_patch_N] + [class_token_pos, image_patch_1_pos, image_patch_2_pos, ..., image_patch_N_pos]
```

**Equations 2&3:**
The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded selfattention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before every block, and residual connections after every block (Wang et al., 2019; Baevski \& Auli, 2019).

In pseudocode:

```python
# Equation 2
x_output_MSA_block = MSA_layer(LN_layer(x_input)) + x_input

# Equation3
x_output_MLP_block = MLP_layer(LN_Layer(x_output_MSA_block)) + x_output_MSA_block
```

**Equation 4:**
Similar to BERT's [class] token, we prepend a learnable embedding to the sequence of embedded patches ( $\mathbf{z}_0^0=\mathbf{x}_{\text {class }}$ ), whose state at the output of the Transformer encoder ( $\mathbf{z}_L^0$ ) serves as the image representation $\mathbf{y}$ (Eq. 4). Both during pre-training and fine-tuning, a classification head is attached to $\mathbf{z}_L^0$. The classification head is implemented by a MLP with one hidden layer at pre-training time and by a single linear layer at fine-tuning time.

* MLP = multilayer perceptron = feed forward neural network with N number of layers.
* MLP = one hidden layer at a time.
* MLP = single line layer at fine-tuning time.

In pseudocode:
```python
# Equation 4
y = MLP(LN_Layer(x_output_MLP_block))
```

### Table 1

![](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/images/08-vit-paper-table-1.png?raw=true)

* ViT-Base, ViT-Large, and ViT Huge are all different sizes of the same model architecture.
* Layers - the number of transformer encoder layers
* Hidden size $D$ - the embedding size throughout the architecture.
* MLP size - the number of hidden units/neurons in the MLP.
* Head - the number of multi-head self-attention.

## 4. Equation 1: Split data into patches and creating the class, position and patch embedding

Layers = input -> function -> output

What's the input shape?

What's the output shape?`

* Input shape: (224, 224, 3) -> Single image -> (height, width, colour_channels)
* Output Shape: ??

### 4.1 Calculate input and output shapes by hand

**Equation 1:**
An overview of the model is depicted in Figure 1. The standard Transformer receives as input a 1D sequence of token embeddings. To handle 2D images, we reshape the image $\mathbf{x} \in \mathbb{R}^{H \times W \times C}$ into a sequence of flattened 2D patches $\mathbf{x}_p \in \mathbb{R}^{N \times\left(P^2 \cdot C\right)}$, where ( $H, W$ ) is the resolution of the original image, $C$ is the number of channels, $(P, P)$ is the resolution of each image patch, and $N=H W / P^2$ is the resulting number of patches, which also serves as the effective input sequence length for the Transformer. The Transformer uses constant latent vector size $D$ through all of its layers, so we flatten the patches and map to $D$ dimensions with a trainable linear projection (Eq. 1). We refer to the output of this projection as the patch embeddings.

**Equation 1:**
Position embeddings are added to the patch embeddings to retain positional information. We use standard learnable 1D position embeddings, since we have not observed significant performance gains from using more advanced 2D-aware position embeddings (Appendix D.4). The resulting sequence of embedding vectors serves as input to the encoder.

* Input image: $H\times{W}\times{C}$ (height x width x color_channels)
* Output Image: $\mathbb{R}^{N \times\left(P^2 \cdot C\right)}$
* H = height
* W = width
* C = color_channels
* P = pathc size
* N = number of patches = (heigth * width) / p^2
* D = constant latent vector size = embedding dimension (see Table 1)

In [ ]:
# Create example values
height = 224
width = 224
colour_channels = 3
patch_size = 16

# Calculate the number of patches
number_of_patches = int((height * width) / patch_size**2)
number_of_patches

In [ ]:
# Input shape
embedding_layer_input_shape = (height, width, colour_channels)

# Output shape
embedding_layer_output_shape = (number_of_patches, patch_size**2 * colour_channels)

print(f"Input shape (single 2D image): {embedding_layer_input_shape}")
print(f"Output shape (single 1D sequence of patches): {embedding_layer_output_shape} -> (number_patches, embedding_dimension)")

### 4.2 Turning a single image into patches

Let's visualize, visualize, visualize!

In [ ]:
# View a single image
plt.imshow(image.permute(1, 2, 0))
plt.title(class_names[label])
plt.axis(False);

In [ ]:
image.shape

In [ ]:
# Get the top row of the image
image_permuted = image.permute(1, 2, 0) # convert images to color channels last  (C, H, W) -> (H, W, C)
print(image_permuted.shape)

# Index to plot the top row of pixels
patch_size = 16
plt.figure(figsize=(patch_size, patch_size))
plt.imshow(image_permuted[:patch_size, :, :])
plt.axis(False)

In [ ]:
# Setup code to plot top row as patches
img_size = 224
patch_size = 16
num_patches = img_size / patch_size
assert img_size % patch_size == 0, "Image size must be divisible by patch size"
print(f"Number of patches per row: {num_patches}\n Patch_size: {patch_size}\n pixels x {patch_size} pixels")

# Create a series of subplots
fig, axs = plt.subplots(nrows = 1,
                        ncols=img_size//patch_size, # one column for each patch
                        sharex=True,
                        sharey=True,
                        figsize=(patch_size, patch_size)
                        )

# Iterate through number of patches in the top row
for i, patch in enumerate(range(0, img_size, patch_size)):
  axs[i].imshow(image_permuted[:patch_size, patch:patch+patch_size, :])
  axs[i].set_xlabel(i+1) # set the patch label
  axs[i].set_xticks([])
  axs[i].set_yticks([])
plt.show()

In [ ]:
# Setup code to plot whole image as patches
img_size = 224
patch_size = 16
num_patches = img_size / patch_size
assert img_size % patch_size == 0, "Image size must be divisilbe by patch size"
print(f"Number of patches per row: {num_patches}")
print(f"Number of patches per column: {num_patches}")
print(f"Number of total patches: {num_patches**2}")
print(f"Patch_size: {patch_size} pixels x {patch_size} pixels")

# Create a series of subplots
fig, axs = plt.subplots(nrows = int(img_size / patch_size),
                        ncols = int(img_size / patch_size),
                        figsize = (num_patches, num_patches),
                        sharex = True,
                        sharey = True)

# Loop through height and width of the image
for i, patch_height in enumerate(range(0, img_size, patch_size)): # iterate through height
  for j, patch_width in enumerate(range(0, img_size, patch_size)): # iterate through width
    # Plot the permuted image on the different axis
    img_plot = image_permuted[patch_height:patch_height+patch_size, # Iterate through the height dim
                                  patch_width: patch_width + patch_size, # Iterate through the width
                                  :] # Get all the colour channels
    axs[i, j].imshow(img_plot)

    # Set up label information for each subplot (patch)
    axs[i, j].set_ylabel(i + 1,
                         rotation="horizontal",
                         horizontalalignment = "right",
                         verticalalignment = "center")
    axs[i, j].set_xlabel(j + 1)
    axs[i, j].set_xticks([])
    axs[i, j].set_yticks([])
    axs[i, j].label_outer()

# Setup a title for the plot
fig.suptitle(f"{class_names[label]} -> Patchified", fontsize = 14)

### 4.3 Creating images patches and turning them into patch embeddings

Perhaps we could create the image patches and image patch embeddings in a single step using `torch.nn.Conv2d()` and setting the kernel size and stride parameters to `patch_size`

In [ ]:
# Create conv2d layer to turn image into patches of learnable feature maps (embeddings)
from torch import nn

# Set the patch size
patch_size = 16

# Create a conv2d layer with hyperparameters from the ViT paper
conv2d = nn.Conv2d(in_channels=3, # for color images
                   out_channels = 768, # D size from Table 1,
                   kernel_size = patch_size,
                   stride = patch_size,
                   padding = 0)
conv2d

In [ ]:
# Visualize single image
plt.imshow(image.permute(1, 2, 0))
plt.title(class_names[label])
plt.axis(False)

In [ ]:
# Pass the image through the convolutional layer
print(image.unsqueeze(0).shape)
image_out_of_conv = conv2d(image.unsqueeze(0)) # add batch dimension -> (batch_size, color_channels, height, width)
print(image_out_of_conv.shape)

In [ ]:
14*14

Now we've passed a signle image to our `conv2d` layer, it's shape is:

```python
torch.Size([1, 768, 14, 14]) # [batch_size, embedding_dim, feature_map_height, feature_map_width]

```

In [ ]:
# Plot random convolutional feature maps
import random
random_indexes = random.sample(range(0, 768-10), k = 5)
print(f"Showing random convolutional feature maps from indexes: {random_indexes}")

# Create plot
fig, axs = plt.subplots(nrows = 1,
                        ncols = 5,
                        figsize = (12, 12))

# Plot random image feature maps
for i, idx in enumerate(random_indexes):
  image_conv_feature_map = image_out_of_conv[:, idx, :, :] # Index on the output tensor of the conv2d layer
  axs[i].imshow(image_conv_feature_map.squeeze().detach().numpy()) # Remove batch dimension, and remove from grad tracking/switch to numpy for matplotlib
  axs[i].set(xticklabels = [], yticklabels = [], xticks = [], yticks = [])

In [ ]:
# Get a single feature map in tensor form
single_feature_map = image_out_of_conv[:, 0, :, :]
single_feature_map, single_feature_map.requires_grad

### 4.4 Flattening the patch embedding with `torch.nn.Flatten()`

Right now we've gotten a series of convolutional feature maps (patch embeddings) that we want to flatten into a sequence of patch embeddings to satisfy the input criteria of the ViT Transformer Encoder.

In [ ]:
print(f"{image_out_of_conv.shape} -> (batch_size, embedding_dim, feature_map_height, feature_map_width)")


Want: (batch_size, number_of_patches, embedding_dim)

In [ ]:
from torch import nn
flatten_layer = nn.Flatten(start_dim = 2,
                           end_dim = 3)
flatten_layer(image_out_of_conv).shape

In [ ]:
# Put everything together
plt.imshow(image.permute(1, 2, 0))
plt.title(class_names[label])
plt.axis(False)
print(f"Original image shape: {image.shape}")

# Turn image into feature maps
image_out_of_conv = conv2d(image.unsqueeze(0)) # Add a batch dimension
print(f"Image feature map shape: {image_out_of_conv.shape}")

# Flatten the feature maps
image_out_of_conv_flattened = flatten_layer(image_out_of_conv)
print(f"Flattened feature map shape: {image_out_of_conv_flattened.shape}")


In [ ]:
# Rearrange of output flattened
image_out_of_conv_flattened_permuted = image_out_of_conv_flattened.permute(0, 2, 1)
print(f"{image_out_of_conv_flattened_permuted.shape} -> (batch_size, number_of_patches, embedding_dim)")

In [ ]:
single_flattened_feature_map = image_out_of_conv_flattened_permuted[:, :, 0] # Select (batch_size, embedding_dim, single_idx_patch)
plt.figure(figsize=(22, 22))
plt.imshow(single_flattened_feature_map.detach().numpy())
plt.title(f"Flattened feature map shape: {single_flattened_feature_map.shape}")
plt.axis(False)


### 4.5 Turning the ViT patch embedding layer into a PyTorch module

We want this module to do a few things:
1. Create a class called `PatchEmbedding` that inherits from `nn.Module`.
2. Initialize with appropiate hyperparameters, such as channels, embedding dimension, patch size.
3. Create a layer to turn an image into embedded patches using `nn.Conv2d()`
4. Create a layer to flatten the feature maps of the output of the layer in 3.
5. Define a `forward()` that defines the forward computatoin (e.g pass through layer from 3 to 4).
6. Make sure the output shape of the layer reflects the required output shape of the patch embedding.

In [ ]:
# 1. Create a class called PatchEmbedding
class PatchEmbedding(nn.Module):
  # 2. Initialize the layer with appropiate hyperparameters
  def __init__(self,
               in_channels:int = 3,
               patch_size:int = 16,
               embedding_dim: int = 768): # from Table 1 for ViT-Base
    """
    Turns a 2D input image into a 1D sequence learnable embeddinf vector.

    Args:
      in_channels (int): number of color channels for the input image. (3 for RGB)
      patch_size (int): Size of patches to convert input image into. Defaults to 16.
      embedding_dim (int): Size of the embedding dimension. Defaults to 768.
    """
    super().__init__()

    # Store all hyperparameters
    self.in_channels = in_channels
    self.patch_size = patch_size
    self.embedding_dim = embedding_dim

    # 3. Create a layer to turn an image into embedded patches
    self.patcher = nn.Conv2d(in_channels=in_channels,
                             out_channels = embedding_dim,
                             kernel_size = patch_size,
                             stride = patch_size,
                             padding = 0)

    # 4. Create a layer to flatten feature map outputs of Conv2d
    self.flatten = nn.Flatten(start_dim=2,
                              end_dim=3)

  # 5. Define the forward method
  def forward(self, x): # Input x is (batch_size, channel_in, height, width) -> (N, 3, 224, 224)
    # Create assertion to check that inputs are the correct shape
    image_resolution = x.shape[-1]
    assert image_resolution % self.patch_size == 0, f"Input image size must be divisible by the patch size, image_shape: {image_resolution}, patch_size: {self.patch_size}"

    # Perform the forward pass
    #print(x.shape)
    x_patched = self.patcher(x)
    #print(x_patched.shape)
    x_flattened = self.flatten(x_patched)
    #print(x_flattened.shape)

    # 6. Make sure the returned sequence embedding dimensions are in the right order (batch_size, num_of_pacthes, embedding_dim)
    return x_flattened.permute(0, 2, 1) # [batch_size, P^2*C, N] -> [batch_size, N, P^2*C]

In [ ]:
set_seeds()

In [ ]:
# Create amn instance of patch embedding layer
patchify = PatchEmbedding(in_channels=3,
                          patch_size = 16,
                          embedding_dim=768)

# Pass a single image through embedding layer
print(f"Input image size: {image.unsqueeze(0).shape}")
patch_embedded_image = patchify(image.unsqueeze(0)) # Add an extra batch dimension
print(f"Output patch embedded image shape: {patch_embedded_image.shape}")

In [ ]:
rand_img_tensor = torch.randn(1, 3, 224, 224)
rand_img_tensor_bad = torch.randn(1, 3, 250, 250)

# Let's try them
#patchify(rand_img_tensor)
#patchify(rand_img_tensor_bad)

### 4.6 Creating a class token embedding

Want to: preprend a learnable class token to the start of the patch embedding.

In [ ]:
patch_embedded_image.shape

In [ ]:
# Get the batch size and embedding dimension
batch_size = patch_embedded_image.shape[0]
embedding_dim = patch_embedded_image.shape[-1]
batch_size, embedding_dim

In [ ]:
# Create a class token embedding as a learnable parameter that shares the same size as the embedding Dimension (D)
class_token = nn.Parameter(torch.ones(batch_size, 1, embedding_dim),
                           requires_grad=True)
class_token.shape

In [ ]:
patch_embedded_image.shape

In [ ]:
# Add the class token embedding to the front of the patch embedding
patch_embedded_image_with_class_embedding = torch.cat((class_token, patch_embedded_image),
                                                       dim = 1) # Number of patches dimension
print(patch_embedded_image_with_class_embedding)
print(f"Class token embedding shape: {class_token.shape}")
print(f"Patch embedding image with class embedding shape {patch_embedded_image_with_class_embedding.shape} -> (batch_size, class_token + number_of_patches, embedding_dim)")

### 4.7 Creating the position embedding

Want to: create a series of 1D learnable position embeddings and to add them to the sequence of patch embeddings.

In [ ]:
# View the sequence of patch embeddings with the prepended class embedding
patch_embedded_image_with_class_embedding, patch_embedded_image_with_class_embedding.shape

In [ ]:
# Calculate the N (number_of_patches)
number_of_patches = int((height * width) / patch_size**2)
number_of_patches

# Get the embedding dimension
embedding_dim = patch_embedded_image_with_class_embedding.shape[-1]
embedding_dim

# Create the learnable 1D position embedding
position_embedding = nn.Parameter(torch.ones(
    batch_size,
    number_of_patches + 1,
    embedding_dim),
    requires_grad=True)
position_embedding, position_embedding.shape

In [ ]:
# Add the position embedding to the patch and class token embedding
patch_and_position_embedding = patch_embedded_image_with_class_embedding + position_embedding
patch_and_position_embedding, patch_and_position_embedding.shape

### 4.8 Putting it all together: from image to embedding

We've written code to turn an image in a flattened sequence of patch embeddings.

Now let's see it all in one cell.

In [ ]:
# Set the seeds
set_seeds()

# Let's see our image
print(f"Original singular image: {image.shape} -> [colour_channels, height, width]") # C x H x W
height, width = image.shape[1], image.shape[2]

# Let's add a batch dimension
image_passed = image.unsqueeze(0)
print(f"Batch added singular image: {image_passed.shape} [batch_size, colour_channels, height, width]")

# Let's create our patch_embedding layer using our custom PatchEmbedding nn.module class
# in_channels = 3
# patch_size = 16
# embedding_dim = 768
patch_size = 16
patchify = PatchEmbedding(in_channels=3,
                          patch_size = patch_size,
                          embedding_dim=768)
patchified_image = patchify(image_passed)
print(f"Patchified image shape: {patchified_image.shape} -> [batch_size, num_patches, embedding_dim]")

# Obtain all the dims from the data
batch_size = patchified_image.shape[0]
num_patches = int((height * width)/patch_size**2)
embedding_dim = patchified_image.shape[-1]

# Now let's add the class token across the num_patches dimension
class_token = nn.Parameter(
    torch.randn(batch_size, 1, embedding_dim),
    requires_grad = True)

# Prepend/Concatanate it with patchified_image to make it [B x N x D] to [B x (N + 1) x D]
patchified_image_with_class_token = torch.cat((class_token, patchified_image),
                                               dim =1)
print(f"Patchified image with class token: {patchified_image_with_class_token.shape} -> [batch_size, num_patches + 1, embedding_dim]")

# Now let's create the position embedding and add it, shape must be [B x (N + 1) x D]
position_embedding = nn.Parameter(
    torch.randn(batch_size, num_patches + 1, embedding_dim),
    requires_grad=True)
print(f"Position embedding shape: {position_embedding.shape} -> [batch_size, num_patches + 1, embedding_dim]")

# Now let's add it up and voila!
patchified_image_with_class_token_and_position_embedding = patchified_image_with_class_token + position_embedding
print(patchified_image_with_class_token_and_position_embedding.shape)

## Equation 2: Multihead Self-attention MSA block

* Multihead self-attention: which part of a sequence should pay the most attention to itself?
  * In our case, we have a series of embedded images, which patch significantly relates to another patch.
  * We want our neural network (ViT) to learn this relationship/representation.
* To replicate MSA in PyTorch we can use: https://docs.pytorch.org/docs/stable/generated/torch.nn.modules.activation.MultiheadAttention.html
* LayerNorm = Layer normalization (LayerNorm) is a technique to normalize the distribution of intermediate layers. It enables smoother gradients, faster training, and better generalization accuracy.
  * Normalization = make everything have the same mean and same standard deviation.
  * In PyTorch = https://docs.pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html
normalizes values over $D$ dimension, in our case, the $D$ dimension is the embedding dimension.
  * When we normalize along the embedding dimension, it's like making all of the staircase the same size.


In [ ]:
class MultiHeadSelfAttentionBlock(nn.Module):
  '''
  Create a multi-head self attention block ("MSA" block for short)
  '''

  def __init__(self,
               embedding_dim:int = 768, # Hidden size D (embedding dimension) from Table 1 for ViT-Base
               num_heads:int = 12, # Heads from Table 1 for ViT-base
               attn_dropout:int = 0): # Heads from Table 1
    super().__init__()

    # Create the norm layer (LN)
    self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

    # Create multihead attention (MSA) layer
    self.multihead_attn = nn.MultiheadAttention(
        embed_dim=embedding_dim,
        num_heads=num_heads,
        dropout=attn_dropout,
        batch_first = True) # is the batch first? (batch, seq, feature) -> (batch, num_of_patches, embedding_dim)

  def forward(self, x):
    x = self.layer_norm(x)
    attn_output, _ = self.multihead_attn(query = x, # query embeddings
                                         key = x, # key embeddings
                                         value = x, # value embeddings
                                         need_weights = False)
    return attn_output

In [ ]:
# Create an instance MSA block
multihead_self_attention_block = MultiHeadSelfAttentionBlock(
    embedding_dim = 768,
    num_heads = 2,
    attn_dropout = 0)

# Pass the patch and position image embedding sequence through MSA block
patched_image_through_msa_block = multihead_self_attention_block(patchified_image_with_class_token_and_position_embedding)
print(f"Patched image with class token and position embedding shape: {patchified_image_with_class_token_and_position_embedding.shape}")
print(f"Patched image through MSA block: {patched_image_through_msa_block.shape}")


In [ ]:
patchified_image_with_class_token_and_position_embedding

In [ ]:
patched_image_through_msa_block

## 6. Equation 3: Multilayer Perception (MLP block)

* **MLP** = The MLP contains two layers with a GELU non-linearity (section 3.1):
  * MLP = a quite broad term for a block with a series of layer(s), layers can be multiple or even only one hidden layer.
  * Layers can mean: fully-connected, sense, linear, feed-forward, all are often similar names for the same thing. In PyTorch, they're often called `torch.nn.Linear`. In TensorFlow is `tf.keras.layers.Dense()`.
  * GELU in PyTorch: https://docs.pytorch.org/docs/stable/generated/torch.nn.GELU.html
  * MLP number of hidden units = MLP Size in Table 1
* **Dropout** = Dropout, when used, is applied after every dense layer except for the qkv-projections and directly after adding positional-to patch embeddings. Hybrid models are trained with the exact setup as their ViT counterparts.
  * Value for Dropout available in Table 3:

In pseudocode:

```python
# MLP
x = linear -> non-linear -> dropout -> linear -> dropout
```

In [ ]:
# 1. Create a class that inherits from nn.Module
class MLPBlock(nn.Module):
  """
  Creates a layer normalized multilayer percepttron block ("MLP block" for short)
  """
  # 2 Initialize the class hyperparameters from Table 1 and Table 3
  def __init__(self,
               embedding_dim:int = 768, # Hidden Size D from Table 1 for ViT-Base
               mlp_size:int = 3072, # MLP size from Table 1 for ViT-Base
               dropout = 0.1): # Dropout from Table 3 for ViT-Base
    super().__init__()

    # Create the norm layer
    self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

    # Create the MLP
    self.mlp = nn.Sequential(
        nn.Linear(in_features=embedding_dim,
                  out_features = mlp_size),
        nn.GELU(),
        nn.Dropout(p = dropout),
        nn.Linear(in_features=mlp_size,
                  out_features = embedding_dim)
    )

  def forward(self, x):
    x = self.layer_norm(x)
    x = self.mlp(x)
    return x
    # return self.mlp(self.layer_norm(x)) # same as above

In [ ]:
# Create an instance MLPBlock
mlp_block = MLPBlock(embedding_dim = 768,
                     mlp_size = 3072,
                     dropout = 0.1)

# Pass output the MSABlock through MLPBlock
patched_image_through_mlp_block = mlp_block(patched_image_through_msa_block)
print(f"patched image through MLP block: {patched_image_through_mlp_block.shape}")
print(f"patched image through MSA block: {patched_image_through_msa_block.shape}")

In [ ]:
patched_image_through_mlp_block

In [ ]:
patched_image_through_msa_block

## 7. Creating the Transformer Encoder

The transformer Encoder is a combination of alternating blocks of MSA (equation 2) and MLP (equation 3).

And there are residual connections between each block.

* Encoder =  turn a sequence into learnable representation.
* Decoder = go from learn representation back to some sort of sequence.
* Residual connections = add a layer(s) input to its subsequent output, this enables the cration of deeper networks
(prevents weights from getting too small).

In pseudocode:

```python
# Transformer Encoder
x_input -> MSA block -> [MSA_block_output + x_input] -> MLP_block -> [MLP_block_output + MSA_block_output + x_input] -> ...
```

In [ ]:
class TransformerEncoderBlock(nn.Module):
  def __init__(self,
               embedding_dim: int = 768, # Hidden size D from table 1, 768 for ViT-Base
               num_heads:int = 12, # from table 1
               mlp_size:int = 3072, # from table 1
               mlp_dropout:int = 0.1, # from table 3
               attn_dropout:int = 0):
    super().__init__()

    # Create MSA block (equation 2)
    self.msa_block = MultiHeadSelfAttentionBlock(embedding_dim=embedding_dim,
                                                 num_heads=num_heads,
                                                 attn_dropout=attn_dropout)

    # Create MLP block (equation 3)
    self.mlp_block = MLPBlock(embedding_dim=embedding_dim,
                            mlp_size=mlp_size,
                            dropout=mlp_dropout)

  def forward(self, x):
    x = self.msa_block(x) + x # residual/skip connection for equation 2
    x = self.mlp_block(x) + x # residual/skip connection for equation 3
    return x

In [ ]:
# Create an instance of TransformerEncoderBlock()
transformer_encoder_block = TransformerEncoderBlock()

# Get a summary using torchinfo summary
summary(model = transformer_encoder_block,
        input_size=(1, 197, 768), # (batch_size, number_of_patches, embedding_dim)
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
        )

### 7.2 Create a custom Transformer Encoder block layer with in-built PyTorch layers

So far we've created a transformer encoder by hand.

But because of how good the Transformer architecture is, PyTorch has implemented ready to use Transformer Encoder layers:
* https://docs.pytorch.org/docs/stable/generated/torch.nn.Transformer.html
* https://docs.pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html
* https://docs.pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html

We can create a Transformer Encoder with pure PyTorch layers.


In [ ]:
# Create the same as above with torch.nn.TransformerEncoderLayer()
torch_transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=768, # Embedding size of table 1
                                                             nhead=12,  # heads from table 1
                                                             dim_feedforward=3072, # MLP size of table 1
                                                             dropout=0.1,
                                                             activation="gelu",
                                                             batch_first=True,
                                                             norm_first=True)
torch_transformer_encoder_layer

In [ ]:
# Get a summary using torchinfo summary
summary(model = torch_transformer_encoder_layer,
        input_size=(1, 197, 768), # (batch_size, number_of_patches, embedding_dim)
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
        )

Why spend all this time recreating the transformer encoder when we could've just made it with a single torch layer? practice!

Practice. Practice. Practice

Now we know how things are implemented behind the scences, we can tweak them if necessary

What are the benefits of using a pre-built PyTorch Layer?
* Less prone to errors (goes through a bunch of testing).
* Potential better performance, speed ups and optimizations (boosts!).

## 8. Putting it all together to create ViT

In [ ]:
# Create a ViT class
class ViT(nn.Module):
  def __init__(self,
               img_size:int = 224, # Table 3 from the ViT paper
               in_channels:int = 3,
               patch_size:int = 16,
               num_transformer_layers:int = 12, # Table 1 for "Layers" for ViT-base
               embedding_dim:int = 768, # Hidden size D from Table 1 for ViT-base
               mlp_size:int = 3072, # Table 1
               num_heads:int = 12, # Table 1
               attn_dropout = 0,
               mlp_dropout:int = 0.1,
               embedding_dropout:int=0.1, # Dropout for patch and position embeddings
               num_classes:int = 1000): # number of classes in our classification problem
    super().__init__()

    # Make an assertion that the image size is compatible with the patch size
    assert img_size % patch_size == 0, f"Image size must be divisible by patch size, image_size: {img_size}, patch_size: {patch_size}"

    # Calculate the number of patches (height, width/patch^2)
    self.num_patches = (img_size * img_size) // patch_size ** 2

    # Create learnable class embedding (needs to go at front of sequence of patch embeddings)
    self.class_embedding = nn.Parameter(data = torch.randn(1, 1, embedding_dim),
                                       requires_grad=True)

    # Create learnable position embedding
    self.position_embedding = nn.Parameter(data = torch.randn(1, self.num_patches + 1, embedding_dim))

    # Create dropout value
    self.embedding_dropout = nn.Dropout(p = embedding_dropout)

    # Create patch embedding layer
    self.patch_embedding = PatchEmbedding(in_channels=in_channels,
                                        patch_size=patch_size,
                                        embedding_dim = embedding_dim)

    # Create the Transformer Encoder Block
    self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim=embedding_dim,
                                                                       num_heads=num_heads,
                                                                       mlp_size=mlp_size,
                                                                       mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])

    # Create classifier head
    self.classifier = nn.Sequential(
        nn.LayerNorm(normalized_shape=embedding_dim),
        nn.Linear(in_features=embedding_dim,
                  out_features=num_classes)
    )

  def forward(self, x):
    # Get the batch size
    batch_size = x.shape[0]

    # Create the class token embedding and expand it to match batch size (equation 1)
    class_token = self.class_embedding.expand(batch_size, -1, -1) # "-1" means to infer the dimensions

    # Create the patch embedding (equation 1)
    x = self.patch_embedding(x)

    # Concat class token embedding and patch embedding (equation 1)
    x = torch.cat((class_token, x), dim = 1) # (batch_size, number_of_patches+1, embedding_dim)

    # Add position embedding to class token and patch embedding
    x = self.position_embedding + x

    # Apply dropout to patch embedding ("directly after adding positional-to patch embeddings")
    x = self.embedding_dropout(x)

    # Pass position and patch embedding to Transformer Encoder Layer (equation 2 & 3)
    x = self.transformer_encoder(x)

    # Put 0th index logit through the classifier (equation 4)
    x = self.classifier(x[:, 0])

    return x

In [ ]:
batch_size = 32
embedding_dim = 768
class_embedding = nn.Parameter(data = torch.randn(1, 1, embedding_dim),
                                       requires_grad=True)
class_embedding_expanded = class_embedding.expand(batch_size, -1, -1)
print(class_embedding.shape)
print(class_embedding_expanded.shape)

In [ ]:
x = torch.randn(size = (1, 197, 768))
x[:, 0].shape

In [ ]:
vit = ViT()
vit

In [ ]:
set_seeds()

# Create a random image tensor with same shape as a single image
random_image_tensor = torch.randn(1, 3, 224, 224)

# Create an instance of our ViT architecture with the number of classes we're working with (pizza, steak, and sushi)
vit = ViT(num_classes=len(class_names))

# Pass the random image tensor through the ViT
print(vit(random_image_tensor))

### 8.1 Getting a visual summary of our ViT model

In [ ]:
from torchinfo import summary

# Get a summary using torchinfo summary
summary(model = ViT(num_classes=len(class_names)),
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
        )

## 9. Setting up training ode for our Custom ViT

We've replicated the ViT architecture, now let's see how it performs on our FoodVision Mini data.

### 9.1 Creating an optimizer

The paper staes it uses the Adam optimizer (section 4, Training and Fine Tuning)
with $B_1$ value of 0.9 and $B_2$ of 0.999 (defaults), and weight decay is 0.1.

Weight decay = Regularization technique -> Prevents overfitting, usually the L2 norm of the weights, to the loss function.

### 9.2 Creating a loss function

The ViT paper doesn't actually mention what loss function they used.

So since it's a multi-classification paper, we'll use the `torch.nn.CrossEntropyLoss()`

In [ ]:
device

In [ ]:
from going_modular.going_modular import engine

set_seeds()

# This breaks if using cpu as the device since model is too big in memory :)))
optimizer = torch.optim.Adam(params=vit.parameters(),
                             lr=1e-3,
                             betas = (0.9, 0.999),
                             weight_decay=0.1)

loss_fn = torch.nn.CrossEntropyLoss()

results = engine.train(model = vit,
                       train_dataloader = train_dataloader,
                       test_dataloader = test_dataloader,
                       epochs = 10,
                       loss_fn = loss_fn,
                       optimizer = optimizer,
                       device = device)


### 9.4 What our training setup is missing

How is our training setup different to the ViT paper?

We've replicated model architecture correctly.

But what was different between our trianing procedure (to get such poor results) and the ViT paper training procedures to get such great results?

The main things our training implementation is missing:

Prevent underfitting:
* Data - our setup uses far less data (225 vs millions)

Prevent overfitting:
* learning rate warmup - start with a low learning rate and increase to a base LR.
* Learning rate decay - as your model gets closer to convergence, start to lower the learning rate.
* Gradient Clipping - prevent gradients from getting too big.

### 9.5 Plotting loss curves for our model

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(results)

## 10. Using a pretrained ViT from `torchvision.models`

Generally in deep learning, if you can use a pretrained model from a large dataset on your own problem. it's often a good place to start.

If you can find a pretrained model and use transfer learning, give it a go, it often achieves great results with little data.

### 10.1 Why use a pretrained model?

* Sometimes data is limited.
* Limited training resources.
* Get better results faster (sometimes)...

In [ ]:
# Cost of a TPUv3 for 30 days
cost = 30*24*8
print(f"Cost of renting TPUv3 for 30 straight days in {cost} USD")

In [ ]:
# The following torch v0.12+ and torchvision 0.13+
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### 10.2 Prepare a pretrained ViT for use with FoodVision Mini (turn it into a feature extractor).

In [ ]:
# Get pretrained models for ViT
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT # "DEFAULT = best availbale"

# Setup a ViT model instance with pretrained weights
pretrained_vit = torchvision.models.vit_b_16(weights = pretrained_vit_weights).to(device)

In [ ]:
# Freeze the base parameters
for parameter in pretrained_vit.parameters():
  parameter.requires_grad = False

# Update the classifier head
set_seeds()
pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)

In [ ]:
from torchinfo import summary

# Get a summary using torchinfo summary
summary(model = pretrained_vit,
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
        )

### 10.3 Preparing data for the pretrained Vit model

When using a pretrained model, you want to make sure your data is formatted in the same way that the model was trained on.

* https://docs.pytorch.org/vision/main/models/generated/torchvision.models.vit_b_16.html


In [ ]:
# Get automatic transforms from pretrained ViT weights
vit_transforms = pretrained_vit_weights.transforms()
vit_transforms

In [ ]:
# Setup dataloaders
from going_modular.going_modular import data_setup
train_dataloader_pretrained, test_dataloader_pretrained, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                     test_dir = test_dir,
                                                                                                     transform = vit_transforms,
                                                                                                     batch_size = 32) # Could set a higher batch size because using a pretrained model

### 10.4 Train feature extractor ViT model

In [ ]:
from going_modular.going_modular import engine

# Create optimizer and loss function
optimizer = torch.optim.Adam(params=pretrained_vit.parameters(),
                             lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the classifier head of pretrained ViT
set_seeds()
pretrained_vit_results = engine.train(model = pretrained_vit,
                                      train_dataloader = train_dataloader_pretrained,
                                      test_dataloader = test_dataloader_pretrained,
                                      loss_fn = loss_fn,
                                      optimizer = optimizer,
                                      epochs = 10,
                                      device = device)

### 10.5 Plot the loss curves of our pretrained ViT feature extractor transformer model

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(pretrained_vit_results)

### 10.6 Save our best performing ViT model

Now we've got a model that performs quite well, how about we save it to file and then check it's filesize

We want to check the filesize because fi we wanted to deploy a model to say a website/mobile application, we may have
limitations on the size of the model we can deploy

E.g a smaller model may be required due to compute restrictions

In [ ]:
# Save the model
from going_modular.going_modular import utils

model_name_str = "08_pretrained_vit_feature_extractor_pizza_steak_sushi.pth"

utils.save_model(model = pretrained_vit,
                target_dir = "models",
                model_name = model_name_str)

In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_vit_model_size = Path("models/" + model_name_str).stat().st_size // (1024 * 1024)
print(f"Pretrained ViT model size: {pretrained_vit_model_size} MB")

Our pretrained ViT gets some of the best results we've seen so far on our FoodVision Mini problem, however, the model size is 11x larger than our next best performing model.

Perhaps the larger model size might cause issues when we go to deploy it (e.h: hard to deploy such a large file/might not make predictions as fast a smaller model).

### 11. Predicting on a custom image

In [ ]:
import requests

# Import function to make predictions on images and plot them
from going_modular.going_modular.predictions import pred_and_plot_image

# Setup custom image path
custom_image_path = image_path / "04-pizza-dad.jpeg"

# Download the image if it doesn't already exist
if not custom_image_path.is_file():
    with open(custom_image_path, "wb") as f:
        # When downloading from GitHub, need to use the "raw" file link
        request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
        print(f"Downloading {custom_image_path}...")
        f.write(request.content)
else:
    print(f"{custom_image_path} already exists, skipping download.")

# Predict on custom image
pred_and_plot_image(model=pretrained_vit,
                    image_path=custom_image_path,
                    class_names=class_names)